# Исследование сервиса Яндекс.Музыка

<div style="border:solid green 2px; padding: 10px">

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На основе данных Яндекс.Музыки необходимо сравнить поведение пользователей из этих городов.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Для исследования взяты данные о треках, прослушанных жителями Москвы и Санкт-Петербурга. О качестве данных ничего не известно. Поэтому перед проверкой гипотез обязательный этап - обзор данных.

Данные будут проверены на ошибки, а так же будет дана оценка их влияния на исследование. На этапе предобработки будут приняты решения, как поступить с выявленными ошибками.
    
    
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

---
_Основная цель работы - применить базовые навыки библиотеки Pandas, приобретенные при первичном знакомстве с Python._


## Обзор данных

В данном разделе будет сформировно первичное предстваление о данных, предоставленных для исследования.




In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/yandex_music_project.csv')  # чтение файла с данными и сохранение в df
df.to_csv('file_name.csv', index=False)

Выведу на экран первые десять строк исследуемой таблицы:

In [3]:
df.head(10)  # получение первых 10 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
df.info()  # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Имеются названия колонок, указанные не на змеином регистре.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. 
Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных


На данном этапе будет изменен стиль в заголовках столбцов, исключены пропуски. Затем проведена проверка данных на дубликаты.

### Стиль заголовков
Выведу на экран названия столбцов:

In [5]:
df.columns  # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведу названия в соответствие с хорошим стилем:
* несколько слов в названии ззапишу в «змеином_регистре»,
* все символы сделаю строчными,
* устраню пробелы.

Для этого переименую колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})  # переименование столбцов

Проверка результата. Для этого ещё раз выведуна экран названия столбцов:

In [7]:
df.columns  # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

Сначала посчитаю, сколько в таблице пропущенных значений. Для этого достаточно двух методов `pandas`:

In [8]:
df.isna().sum()  # подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для выполняемой работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменю пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создам список `columns_to_replace`, переберу его элементы циклом `for` и для каждого столбца выполню замену пропущенных значений:

In [9]:
columns_to_replace = ['track', 'artist', 'genre']  # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')


Необходимо убедиться, что в таблице не осталось пропусков. Для этого ещё раз посчитаю пропущенные значения.

In [10]:
df.isna().sum()  # подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

Посчитаю явные дубликаты в таблице одной командой:

In [11]:
df.duplicated().sum()  # подсчёт явных дубликатов 

3826

Вызову специальный метод `pandas`, чтобы удалить явные дубликаты:

In [12]:
df = df.drop_duplicates().reset_index(drop=True)  # удаление явных дубликатов (с удалением старых индексов и формированием новых)

Ещё раз посчитаю явные дубликаты в таблице — это необходимо, чтобы убедиться, что полностью от них избавились:

In [13]:
df.duplicated().sum()  # проверка на отсутствие дубликатов 



0

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведу на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлеку нужный столбец датафрейма, 
* применю к нему метод сортировки,
* для отсортированного столбца вызову метод, который вернёт уникальные значения из столбца.

In [14]:
df['genre'].sort_values().unique()  # Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Просмотрю список, чтобы найти неявные дубликаты названия. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Выявлены следующие неявные дубликаты: 
* *hip*,
* *hop*,
* *hip-hop*,
* *hiphop*, 
* *africa*, 
* *afrikaans*, 
* *electronic*,
* *электроника*, 
* *türk*, 
* *türkçe*, 
* *latin*, 
* *latino*, 
* *tango*,
* *argentinetango*,
* *tech*, 
* *techno*.

Чтобы очистить от них таблицу, объявлю функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [15]:
# Функция для замены неявных дубликатов

def replace_wrong_genres(wrong_genres, correct_genre):  
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre) 

Вызову `replace_wrong_genres()` и передам ей такие аргументы, чтобы она устранила неявные дубликаты: например вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [16]:
# Устранение неявных дубликатов
duplicates_hiphop = ['hip', 'hop', 'hip-hop'] 
name_hiphop = 'hiphop' 

duplicates_african = ['africa', 'afrikaans']
name_african = 'african'

duplicates_electronic = ['электроника']
name_electronic = 'electronic'

duplicates_turk = ['türk', 'türkçe']
name_turk = 'turk'

duplicates_tango = ['argentinetango']
name_tango = 'tango'

duplicates_techno = ['tech']
name_techno = 'techno'

replace_wrong_genres(duplicates_hiphop, name_hiphop)
replace_wrong_genres(duplicates_african, name_african)
replace_wrong_genres(duplicates_electronic, name_electronic)
replace_wrong_genres(duplicates_turk, name_turk)
replace_wrong_genres(duplicates_tango, name_tango)
replace_wrong_genres(duplicates_techno, name_techno)

Выполню проверку, что заменила неправильные названия:

* *hip*,
* *hop*,
* *hip-hop*,
* *hiphop*, 
* *africa*, 
* *afrikaans*, 
* *electronic*,
* *электроника*, 
* *türk*, 
* *türkçe*, 
* *latin*, 
* *latino*, 
* *tango*,
* *argentinetango*,
* *tech*, 
* *techno*.

Выведу отсортированный список уникальных значений столбца `genre`:

In [17]:
df['genre'].sort_values().unique()  # Проверка на неявные дубликаты 

array(['acid', 'acoustic', 'action', 'adult', 'african', 'alternative',
       'alternativepunk', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'art', 'audiobook', 'author',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'chanson', 'children', 'chill', 'chinese', 'choral', 'christian',
       'christmas', 'classical', 'classicmetal', 'club', 'colombian',
       'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Вы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения вы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверю это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделю пользователей Москвы и Санкт-Петербурга
* Сравню, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [18]:
df.groupby('city')['user_id'].count()  # Подсчёт прослушиваний в каждом городе


city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппирую данные по дню недели и подсчитаю прослушивания в понедельник, среду и пятницу. _Необходимо учесть, что в данных есть информация о прослушиваниях только за эти дни._


In [19]:
df.groupby('day')['time'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Ранее было показано, как работает группировка по городу и по дням недели. Теперь необходимо задать функцию, которая объединит два эти расчёта.

Создам функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраню в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применю последовательную фильтрацию с логической индексацией.

Затем посчитаю значения в столбце `user_id` получившейся таблицы. Результат сохраню в новую переменную и верну эту переменную из функции.

In [20]:
# <создание функции number_tracks()>
def number_tracks(day, city):  
    track_list = df[df['day'] == day] 
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count
# Объявляется функция с двумя параметрами: day, city.
# В переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city (используйте последовательную фильтрацию
# с помощью логической индексации).
# В переменной track_list_count сохраняется число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.
# Функция возвращает число - значение track_list_count.

# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id. 
# Это количество функция вернёт в качестве результата

Вызову `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [21]:
mos_mon = number_tracks('Monday', 'Moscow')
display(mos_mon)  # количество прослушиваний в Москве по понедельникам

15740

In [22]:
spb_mon = number_tracks('Monday', 'Saint-Petersburg')
display(spb_mon)  # количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [23]:
mos_wed = number_tracks('Wednesday', 'Moscow')
display(mos_wed)  # количество прослушиваний в Москве по средам

11056

In [24]:
spb_wed = number_tracks('Wednesday', 'Saint-Petersburg')
display(spb_wed)  # количество прослушиваний в Санкт-Петербурге по средам

7003

In [25]:
mos_fri = number_tracks('Friday', 'Moscow')
display(mos_fri)  # количество прослушиваний в Москве по пятницам

15945

In [26]:
spb_fri = number_tracks('Friday', 'Saint-Petersburg')
display(spb_fri)  # количество прослушиваний в Санкт-Петербурге по пятницам

5895

Создам c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [27]:
data = [
    ['Moscow', mos_mon, mos_wed, mos_fri], 
    ['Saint-Petersburg', spb_mon, spb_wed, spb_fri]  # Таблица с результатами
]
columns = ['city', 'monday', 'wednesday', 'friday']
display(pd.DataFrame(data = data, columns = columns))

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраню таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [28]:
moscow_general = df[df['city'] == 'Moscow']  # получение таблицы moscow_general из тех строк таблицы df, 
display(moscow_general)  # для которых значение в столбце 'city' равно 'Moscow'


,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [29]:
spb_general =  df[df['city'] == 'Saint-Petersburg'] # получение таблицы spb_general из тех строк таблицы df,
display(spb_general)  # для которых значение в столбце 'city' равно 'Saint-Petersburg'


,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
...,...,...,...,...,...,...,...
61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Saint-Petersburg,21:14:40,Monday
61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Saint-Petersburg,21:06:50,Monday
61241,29E04611,Bre Petrunko,Perunika Trio,world,Saint-Petersburg,13:56:00,Monday
61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Saint-Petersburg,09:22:13,Monday


Объявлю функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [30]:
def genre_weekday(table, day, time1, time2):   # Объявление функции genre_weekday() с параметрами table, day, time1, time2,
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['time'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False).head(10)
    return genre_df_sorted

    
  # функция, которая возвращает информацию о самых популярных жанрах в указанный день в
  # заданное время:
  # 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
  #    которых одновременно:
  #    - значение в столбце day равно значению аргумента day
  #    - значение в столбце time больше значения аргумента time1
  #    - значение в столбце time меньше значения аргумента time2
  #    Используйте последовательную фильтрацию с помощью логической индексации.
  # 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
  #    столбцов и посчитать методом count() количество записей для каждого из
  #    присутствующих жанров, получившийся Series записать в переменную
  #    genre_df_count
  # 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
  #    в переменную genre_df_sorted
  # 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
  #    популярных жанров (в указанный день, в заданное время)

Cравню результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [31]:
display(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))  # вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
  # объекты, хранящие время, являются строками и сравниваются как строки
  # пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: time, dtype: int64

In [32]:
display(genre_weekday(spb_general, 'Monday', '07:00', '11:00'))  # вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: time, dtype: int64

In [33]:
display(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))  # вызов функции для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: time, dtype: int64

In [34]:
display(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))  # вызов функции для вечера пятницы в Петербурге

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: time, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппирую таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортирую результат в порядке убывания и сохраню его в таблице `moscow_genres`.

In [35]:
moscow_general = moscow_general.groupby('genre')['time'].count()  # одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
moscow_genres = moscow_general.sort_values(ascending=False)  # подсчёт числа значений 'genre' в этой группировке методом count(), 
  # сортировка получившегося Series в порядке убывания и сохранение в moscow_genres

Выведу на экран первые десять строк `moscow_genres`:

In [36]:
display(moscow_genres.head(10))  # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: time, dtype: int64

Теперь повторюто же и для Санкт-Петербурга.

Сгруппирую таблицу `spb_general` по жанру. Посчитаю прослушивания треков каждого жанра. Результат отсортирую в порядке убывания и сохраню в таблице `spb_genres`:


In [37]:
spb_general = spb_general.groupby('genre')['time'].count()  # одной строкой: группировка таблицы spb_general по столбцу 'genre', 
spb_genres = spb_general.sort_values(ascending=False)  # подсчёт числа значений 'genre' в этой группировке методом count(), 
  # сортировка получившегося Series в порядке убывания и сохранение в spb_genres

Выведу на экран первые десять строк `spb_genres`:

In [38]:
display(spb_genres.head(10))  # просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: time, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

<div style="border:solid black 2px; padding: 10px">

Были проверены три гипотезы и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез показали, насколько они достоверны, исходя из имеющихся данных. Но это предмет следующих исследований.
